In [20]:
import pandas as pd
from pandas.io.json import json_normalize
from flatten_json import flatten_json
import json
import folium
import geopy
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim

from os import environ
import requests
import urllib2

In [21]:
#Download File
response = urllib2.urlopen('https://www.bundestag.de/blob/189476/1a4f5aa65b51978e4d4ee0b465ef4990/lobbylisteaktuell-data.pdf')
html = response.read()
#print html

In [22]:
#Get Data from Lobbyliste
LobbyL = open("/Users/Waspish/Documents/OKF/Datenschule/scraper-lobbyliste/listetest/lobbyliste.20160812.json")
LobbyLr = LobbyL.read()
LobbyL_json = json.loads(LobbyLr)
LobbyL_flattend = [flatten_json(l) for l in LobbyL_json]

LobbyL_flattend 



[{'addr_0_addr': 'Hinter der katholischen Kirche 3',
  'addr_0_city': 'Berlin',
  'addr_0_country': 'Deutschland',
  'addr_0_email': 'schimmel@1219.eu',
  'addr_0_fax': '+493051057785',
  'addr_0_name': u'1219. Deutsche Stiftung fr interreligisen und interkulturellen Dialog e. V.',
  'addr_0_postcode': '10117',
  'addr_0_tel': '+493051057773',
  'addr_0_type': 'main',
  'addr_0_www': 'http://www.1219.eu',
  'addr_1_addr': u'c/o Missionszentrale der Franziskaner, Albertus-Magnus-Strae 39',
  'addr_1_city': 'Bonn',
  'addr_1_country': 'Deutschland',
  'addr_1_fax': '+492289535440',
  'addr_1_postcode': '53177',
  'addr_1_tel': '+49228953540',
  'addr_1_type': 'extra',
  'description': u'Interreligiser und interkultureller Dialog: \n- wissenschaftliche Bearbeitung von Aspekte des Dialogs und der \nFunktion von Religion in der deutschen und europischen Gesellschaft; \n- Eintreten fr das Grundrecht und das universale Menschenrecht Religionsfreiheit(auchdienegativeReligionsfreiheit/Weltan- \

In [23]:
#Dataframe of Lobbylist
Lobbydf = pd.DataFrame(LobbyL_flattend).head(10)
Lobbydf_clean = Lobbydf[["addr_0_name","addr_0_addr","addr_0_postcode","addr_0_www"]]
Lobbydf_clean.columns = ['organisation', 'address','postcode','website']
Lobbydf_clean

#Lobbydf_clean.organisation.str.cat(sep=',')

#Lobbydf_clean['period'] = Lobbydf_clean[['address', 'postcode']].apply(lambda x: ''.join(x), axis=1)

df = Lobbydf_clean.address.str.cat(Lobbydf_clean.postcode)

df



0                Hinter der katholischen Kirche 310117
1                        Unter den Linden 19 - 2310117
2                                Pariser Platz 4a10117
3    c/o HST Hanse StiftungsTreuhand GmbH, Poststra...
4                           Robert-Bosch-Strae 1064331
5                                 Georgenstrae 2310117
6                          Neuer Jungfernstieg 2120354
7                                    Ackerstrae 310115
8                              Stresemannallee 3060596
9                         Adenauerallee 212 - 21453113
Name: address, dtype: object

In [28]:
#Get Geodata
API_KEY = 'search-hovxP48'
BASE_URL = 'https://search.mapzen.com/v1/search'

Cordinates = []
for a in df:
    print a
    resp = requests.get(BASE_URL, params = {'api_key': API_KEY, 'size': 1, 'text':a})
    data = resp.json()
    bbox = data['bbox']
    print bbox
    lat = (bbox[1] + bbox[3]) / 2
    lng = (bbox[0] + bbox[2]) / 2
    cor = [lat,lng]
    Cordinates.append(cor) 
    #pt = {'longitude': (bbox[0] + bbox[2]) / 2, 'latitude': (bbox[1] + bbox[3]) / 2  }
    #print pt


print Cordinates
'''

print(lat)'''


Hinter der katholischen Kirche 310117
[13.395608, 52.515964, 13.395608, 52.515964]
Unter den Linden 19 - 2310117
[10.110504, 53.382874, 10.110504, 53.382874]
Pariser Platz 4a10117
[6.967401, 49.247066, 6.967401, 49.247066]
c/o HST Hanse StiftungsTreuhand GmbH, Poststrae 5120354
[11.001649, 53.905443, 11.002096, 53.905793]
Robert-Bosch-Strae 1064331


KeyError: 'bbox'

In [26]:
#Set Coordinates for Berlin
Latitude= 52.52
Longitude= 13.41
B_COORDINATES = (Latitude,Longitude)

#Set Map for Berlin
Berlin = folium.Map(location=B_COORDINATES, zoom_start=10)

print(lat,lng)


(53.905618000000004, 11.001872500000001)


In [31]:
#marker_cluster = folium.MarkerCluster("Vereine").add_to(Berlin)
'''for a in Cordinates:
    folium.Marker(a,popup=org,icon=folium.Icon(color='green',icon='info-sign')).add_to(Berlin)
    '''

marker_cluster = folium.MarkerCluster("Vereine").add_to(Berlin)

for a in Cordinates: 
    folium.Marker(a,popup="Verein").add_to(marker_cluster)
    

Berlin

